In [37]:
library("magrittr")
library("dplyr")
library("reshape2")

In [90]:
d <- Titanic %>% as.data.frame
names(d) <- tolower(names(d))
glimpse(d)
dcast(d, class+sex+age~survived, value.var = "freq") %>%
  mutate(all=Yes+No) %>%
  select(-No)-> d.train
head(d.train)

Observations: 32
Variables:
$ class    (fctr) 1st, 2nd, 3rd, Crew, 1st, 2nd, 3rd, Crew, 1st, 2nd, 3rd, ...
$ sex      (fctr) Male, Male, Male, Male, Female, Female, Female, Female, M...
$ age      (fctr) Child, Child, Child, Child, Child, Child, Child, Child, A...
$ survived (fctr) No, No, No, No, No, No, No, No, No, No, No, No, No, No, N...
$ freq     (dbl) 0, 0, 35, 0, 0, 0, 17, 0, 118, 154, 387, 670, 4, 13, 89, 3...


,class,sex,age,Yes,all
1,1st,Male,Child,5,5
2,1st,Male,Adult,57,175
3,1st,Female,Child,1,1
4,1st,Female,Adult,140,144
5,2nd,Male,Child,11,11
6,2nd,Male,Adult,14,168


In [156]:
model.glm <- glm(Yes~. , data=d.train, family = "gaussian")
summary(model.glm)
model.glm <- glm(Yes~. , data=d.train, family = poisson)
summary(model.glm)
model.glm <- glm(cbind(Yes,all)~. , data=d.train, family = "binomial")
summary(model.glm)


Call:
glm(formula = Yes ~ ., family = "gaussian", data = d.train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-36.400  -12.414   -3.521   19.413   49.721  

Coefficients:
             Estimate Std. Error t value Pr(>|t|)   
(Intercept)   7.38459   18.23216   0.405  0.69491   
class2nd    -19.17653   20.95846  -0.915  0.38406   
class3rd    -25.99979   21.46890  -1.211  0.25672   
classCrew   -26.77857   22.05274  -1.214  0.25553   
sexFemale    29.80806   16.71755   1.783  0.10826   
ageAdult     23.22888   19.17829   1.211  0.25666   
all           0.20735    0.04913   4.220  0.00224 **
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 878.0316)

    Null deviance: 47195.9  on 15  degrees of freedom
Residual deviance:  7902.3  on  9  degrees of freedom
AIC: 160.64

Number of Fisher Scoring iterations: 2



Call:
glm(formula = Yes ~ ., family = poisson, data = d.train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-5.0847  -1.2537   0.2397   1.0380   3.8043  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)  1.6937033  0.1678947  10.088  < 2e-16 ***
class2nd    -0.4113725  0.1164011  -3.534 0.000409 ***
class3rd    -0.6006818  0.1090642  -5.508 3.64e-08 ***
classCrew   -1.4290016  0.1816405  -7.867 3.63e-15 ***
sexFemale    1.1203772  0.1167154   9.599  < 2e-16 ***
ageAdult     1.4866289  0.1530904   9.711  < 2e-16 ***
all          0.0040388  0.0003111  12.983  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 945.12  on 15  degrees of freedom
Residual deviance:  77.96  on  9  degrees of freedom
AIC: 162.65

Number of Fisher Scoring iterations: 5



Call:
glm(formula = cbind(Yes, all) ~ ., family = "binomial", data = d.train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.5023  -0.3614   0.0000   0.6117   2.4427  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -0.6201     0.2125  -2.918  0.00353 ** 
class2nd     -0.4451     0.1486  -2.996  0.00274 ** 
class3rd     -0.7624     0.1305  -5.844  5.1e-09 ***
classCrew    -0.3133     0.1346  -2.328  0.01991 *  
sexFemale     1.2010     0.1064  11.289  < 2e-16 ***
ageAdult     -0.5581     0.1841  -3.031  0.00244 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 250.362  on 13  degrees of freedom
Residual deviance:  28.124  on  8  degrees of freedom
AIC: 104.53

Number of Fisher Scoring iterations: 4


In [92]:
model.glm.seleted <- glm(cbind(Yes,all)~class+sex+age , data=d.train, family = "binomial")
AIC(model.glm.seleted)

# model.glm.seleted <- glm(cbind(Yes,all)~class:sex+age , data=d.train, family = "binomial")
# AIC(model.glm.seleted) # --> 101.08

[1] 104.5332

In [124]:
cbind(d.train,res) %>% arrange(desc(res))

,class,sex,age,Yes,all,res
1,2nd,Female,Child,13,13,0.6034481
2,1st,Male,Child,5,5,0.5027565
3,1st,Female,Adult,140,144,0.4930548
4,1st,Female,Child,1,1,0.4862177
5,Crew,Female,Adult,20,23,0.4651163
6,Crew,Female,Child,0,0,0.4583164
7,2nd,Female,Adult,80,93,0.4468806
8,Crew,Male,Child,0,0,0.4089378
9,2nd,Male,Child,11,11,0.3777432
10,3rd,Female,Adult,76,165,0.3263987


## 変数選択(stepAIC)

In [125]:
library("MASS")


Attaching package: ‘MASS’

The following object is masked from ‘package:dplyr’:

    select



In [158]:
model.glm.pow <- glm(cbind(Yes,all)~. , data=d.train, family = "binomial")
stepAIC(model.glm.pow, scope=list(upper=~.^3))


Start:  AIC=104.53
cbind(Yes, all) ~ class + sex + age

            Df Deviance    AIC
+ sex:age    1   20.792  99.20
+ class:sex  3   18.679 101.09
+ class:age  2   23.510 103.92
<none>           28.124 104.53
- age        1   36.951 111.36
- class      3   64.149 134.56
- sex        1  158.979 233.39

Step:  AIC=99.2
cbind(Yes, all) ~ class + sex + age + sex:age

            Df Deviance     AIC
+ class:sex  3   10.766  95.174
+ class:age  2   16.002  98.411
<none>           20.792  99.200
- sex:age    1   28.124 104.533
- class      3   56.023 128.432

Step:  AIC=95.17
cbind(Yes, all) ~ class + sex + age + sex:age + class:sex

            Df Deviance     AIC
+ class:age  2   4.8447  93.253
<none>          10.7657  95.174
- class:sex  3  20.7916  99.200
- sex:age    1  18.6788 101.087

Step:  AIC=93.25
cbind(Yes, all) ~ class + sex + age + sex:age + class:sex + class:age

                Df Deviance     AIC
+ class:sex:age  2   0.0000  92.409
<none>               4.8447  93.253
- clas


Call:  glm(formula = cbind(Yes, all) ~ class + sex + age + sex:age + 
    class:sex + class:age + class:sex:age, family = "binomial", 
    data = d.train)

Coefficients:
                 (Intercept)                      class2nd  
                   9.397e-16                     1.742e-16  
                    class3rd                     classCrew  
                  -1.306e+00                    -3.800e-01  
                   sexFemale                      ageAdult  
                   1.782e-15                    -1.122e+00  
          sexFemale:ageAdult            class2nd:sexFemale  
                   1.094e+00                    -3.360e-15  
          class3rd:sexFemale           classCrew:sexFemale  
                   5.113e-01                     2.684e-01  
           class2nd:ageAdult             class3rd:ageAdult  
                  -1.363e+00                     6.099e-01  
          classCrew:ageAdult   class2nd:sexFemale:ageAdult  
                          NA        

In [161]:
model.glm.pow <- glm(Yes~. , data=d.train, family = "poisson")
stepAIC(model.glm.pow, scope=list(upper=~.^3))


Start:  AIC=162.65
Yes ~ class + sex + age + all

            Df Deviance    AIC
+ class:age  3    33.17 123.86
+ class:all  3    37.67 128.37
+ age:all    1    45.35 132.05
+ sex:age    1    56.93 143.63
+ sex:all    1    70.79 157.48
+ class:sex  3    68.20 158.89
<none>            77.96 162.65
- class      3   161.15 239.85
- sex        1   180.67 263.36
- age        1   193.92 276.62
- all        1   323.26 405.96

Step:  AIC=123.86
Yes ~ class + sex + age + all + class:age

            Df Deviance    AIC
+ class:all  3    3.701 100.39
+ sex:age    1   14.185 106.88
+ class:sex  3   16.675 113.37
+ age:all    1   26.818 119.51
+ sex:all    1   27.789 120.48
<none>           33.166 123.86
- class:age  3   77.960 162.65
- sex        1  127.106 215.80
- all        1  251.062 339.76

Step:  AIC=100.39
Yes ~ class + sex + age + all + class:age + class:all

            Df Deviance     AIC
- sex        1    3.702  98.396
+ sex:all    1    0.792  99.485
<none>            3.701 100.395
+ se


Call:  glm(formula = Yes ~ class + age + all + class:age + class:all, 
    family = "poisson", data = d.train)

Coefficients:
       (Intercept)            class2nd            class3rd           classCrew  
          1.183250            1.579805            1.421301          -20.485835  
          ageAdult                 all   class2nd:ageAdult   class3rd:ageAdult  
          7.898391           -0.028764           -4.122109           -6.164061  
classCrew:ageAdult        class2nd:all        class3rd:all       classCrew:all  
         14.337923            0.005562            0.028717            0.031460  

Degrees of Freedom: 15 Total (i.e. Null);  4 Residual
Null Deviance:	    945.1 
Residual Deviance: 3.702 	AIC: 98.4

In [162]:
# ?glm